# SOME BACKGROUND

~ STEP 1 :- First download the latest Python Veriosn 
            https://www.python.org/downloads/

Please Install the downloaded file on your computer using custom Installation settings. 

~ STEP 2 :- After the installation is done , Open  , In MAC - Terminal or WIN - Terminal , and type "Python3 --version"
 for MAC or "Python --version" for WIN and press Enter 

 If , the Terminal , returns with the verion of Python like "Python 3.11.1" , Installation has been succesful , and then we can proceed further , if not , please check if STEP 1 has been accomplished

~STEP 3 :- After we are done  with the STEP 2 , I am using the Jupytr Lab for the tutorials. , type in terminal "pip install jupyter lab" , this will install the latest verison of Jupyter Lab on your system , automatically you will be redirected to the browser 

~ STEP 4 :- To check , if the installation of Jupyter lab has been successful , type "jupyter lab --version" on Terminal. If , the Terminal , returns with the verion of Jupyter Lab like "4.1.2" , Installation has been succesful , and then we can proceed further , if not , please check if STEP 3 has been accomplished

~ STEP 5 :- A library is a collection of code that makes everyday tasks more efficient. For my tutorial i would recommend Installing pandas , numpy - For data analysis , seaborn , matplotlib - For Visualisation.  

Also , i would be Installing a library called "peakutils" , this woulbe be useful for recogisation of Peaks from the Spectra. https://peakutils.readthedocs.io/en/latest/


~ STEP 6 :- For installing the library , go to terminal , and type "pip install pandas" and after installing . Similarly , do this for numpy , seaborn , matplotlin and PeakUtils

~ STEP 7 :- Now , to open the Jupyter Lab seession in the browser , go to terminal and type "jupyter lab" , you will be automatically redirected to the browser in a jupyter lab session. This is where you will start programming . 

NOTE :- To exit the session , press "Ctrl + C" on the termianal , this will end the running session on your system

~ STEP 8 :- In jupyter lab , your lines of codes or chunk of lines of codes  are called as "Cell" , If the "Cell are of type Markdown - Used for Comments / Information " and If the "Cells are of type Code - Used for Programming" 

You can get more in fo here -  https://www.datacamp.com/tutorial/installing-jupyter-notebook


# Importing the Useful librararies. 

Line [1] is completely optional , This is used to Hide the the Warnings in the Output cell. It's recommended to skip the line[1] , and proceed with the code from line[2].

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show

from bokeh.io import output_notebook
output_notebook()


Loading BokehJS ...

# LOADING THE SPECTRA FILE WHICH WE HAVE

In [16]:
file_path = 'hLIBS_Spectra/CALIBRATION/Batch_2/sample_1/sample1_1.csv'


Sample_Spectra = pd.read_csv(file_path)
Sample_Spectra = Sample_Spectra.rename({' intensity':'Intensities'},axis=1)


Sample_Spectra_Plot = figure(title = 'Polished CR240LA - handheldLIBS - First Measurement - 5 cleaning pulses Depth' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Plot.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="blue" )
Sample_Spectra_Plot.width = 800
Sample_Spectra_Plot.height = 400


print (Sample_Spectra)
show (Sample_Spectra_Plot)



       wavelength  Intensities
0      180.000000          0.0
1      180.033333          0.0
2      180.066667          0.0
3      180.100000          0.0
4      180.133333          0.0
...           ...          ...
23426  960.866667          0.0
23427  960.900000          0.0
23428  960.933333          0.0
23429  960.966667          0.0
23430  961.000000          0.0

[23431 rows x 2 columns]


##### Now , as we see in the figure avove , the spectra have lots of lines , which are very specific to the element present in the Sample . 

Each element has an specific , wavelength associated with it.

Now , i have visited the NIST LIBS database , and download the spectra for all the element in the Periodic Table. The File are updated with that of NIST database as of 17 April 2024.

# CHOOSING THE ELEMENT OF INTEREST

In [4]:

element = 'Zn' # ENTER YOUR ELEMENT OF CHOICE
Reference_Spectra = pd.read_csv(r"NIST/" + element +".csv" , encoding = 'windows - 1252' , usecols=(0,1))
print(Reference_Spectra)

Reference_Spectra_Plot = figure(title = 'NIST Reference Library ', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Reference_Spectra_Plot.line(Reference_Spectra.Wavelength,Reference_Spectra.Spectra , line_width = 2, color ="blue" )
Reference_Spectra_Plot.width = 1200
Reference_Spectra_Plot.height = 600

show(Reference_Spectra_Plot)

      Wavelength       Spectra
0         201.31  1.140000e-05
1         201.35  1.230000e-04
2         201.40  2.160000e-03
3         201.44  1.960000e-02
4         201.48  1.640000e-01
...          ...           ...
1906      780.29  8.790000e-10
1907      780.33  4.820000e-10
1908      780.38  2.260000e-10
1909      780.42  1.220000e-10
1910      780.46  6.590000e-11

[1911 rows x 2 columns]


# APPROACH FOR PEAK DETECTION IN THE NIST ELEMENT SPECTRA

Now, uptil now , we have selected the Material Sample ,which we want to Interact with and the data is stored in variable "Sample_Spectra"
Also , we have choosen the element , which we want to detect in this 'Sample_Spectra' , and the spectral information for that element is stored in variable "Reference_Spectra" 

Now , from this 'Reference_Spectra' , Let's try to get all the Peaks this Spectra has

I have used a library called as Peak_Utils 

https://peakutils.readthedocs.io/en/latest/reference.html

The detail of the function is given below. 

---------------------------------------------------------------------------------------------------------------------------

NOTE :- Now if we keep the minimum_distance_1 = 0 and threshold_1 = 0 , we get all the peaks in the spectra , Tuning of these parameter are most important as per your signal.

Setting the value in parameter 'minimum_distance_1' , will allows to skip the peaks , which are very close to each other. 
Setting the value in parameter 'threshold_1', will allows to skip the peaks with low intensity and also seperate the Peaks , with could be the noise signal. 

So , its important to choose the value's as per your convenience 

----------------------------------------------------------------------------------------------------------------------------

In [5]:
minimum_distance_1 = 0.01 
threshold_1 = 0 #looking for a value above 0.01% , so that Peak with very low Intensities can be ignored 

Reference_Peak = indexes(Reference_Spectra.Spectra , min_dist=minimum_distance_1 , thres=threshold_1) 

Selected_Reference_Spectra = Reference_Spectra.iloc[Reference_Peak] #from the indices , lets try to find the Wavelength and Spectra. 

Reference_Spectra_Peaks = figure(title = 'Peaks in NIST Spectra ', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Reference_Spectra_Peaks.line(Reference_Spectra.Wavelength,Reference_Spectra.Spectra , line_width = 2, color ="blue" )
Reference_Spectra_Peaks.circle(Selected_Reference_Spectra.Wavelength , Selected_Reference_Spectra.Spectra , color = 'red' , size = 8)

Reference_Spectra_Peaks.width = 1200
Reference_Spectra_Peaks.height = 600

print(Selected_Reference_Spectra)
show(Reference_Spectra_Peaks)



      Wavelength       Spectra
29        202.55  2.160000e+11
88        206.20  1.100000e+11
153       209.99  1.170000e+09
219       213.85  7.000000e+10
285       250.21  2.690000e+08
357       255.77  5.090000e+08
387       257.05  8.900000e+06
471       328.22  5.140000e+08
519       330.26  1.500000e+09
616       334.51  2.540000e+09
735       491.18  5.450000e+06
763       492.38  1.080000e+07
919       602.11  9.100000e+05
1091      610.23  1.100000e+06
1268      636.21  1.240000e+08
1466      758.86  3.140000e+06
1522      761.25  1.880000e+05
1740      773.23  1.470000e+06
1800      775.79  3.600000e+05


# Wavelengths for Detected Peak(red circles above)

In [6]:
#Database 1
Reference_Spectra_List = []

# for iteam in Selected_Reference_Spectra.Wavelength:
#     iteam = float(iteam)
#     if iteam > 187.0:        #Only those iteam will be selected , whose wavelength would be greater than 188 nm ,  as our Handheld LIBS range starts from 188nm
#         Reference_Spectra_List.append(iteam)
    

############################################################################################
#Database 2

# New_Reference_Spectra = pd.read_csv(r"NIST_2/" + element +".csv" , encoding = 'windows - 1252' , usecols=(0,1))
# print(New_Reference_Spectra)

# Reference_Spectra_List = (New_Reference_Spectra['Wavelength'] / 10).tolist()


############################################################################################
#Database 3

NISTDB_Reference_Spectra = pd.read_csv(r"NIST_3/" + element +".csv" , encoding = 'windows - 1252' , usecols=(3,4,5,6,7,8))
print(NISTDB_Reference_Spectra)

Reference_Spectra_List = (NISTDB_Reference_Spectra['obs_wl_air(nm)']).tolist()

#############################################################################################
print(Reference_Spectra_List)
Length_Reference_Spectra_List = (len(Reference_Spectra_List))   #Used for Calculating Percentage Detection

     obs_wl_air(nm)  unc_obs_wl ritz_wl_air(nm)  unc_ritz_wl    intens  \
0        213.857351    0.000010      213.857354     0.000009  80000000   
1        334.501339    0.000022      334.501340     0.000021   2900000   
2        208.695000    0.003000        208.6941     0.002200   1600000   
3        330.258289    0.000022      330.258290     0.000016   1400000   
4        260.855756    0.000014      260.855754     0.000013   1300000   
..              ...         ...             ...          ...       ...   
141      407.814000    0.010000        407.813+     0.024000        40   
142      278.281000    0.010000        278.281+     0.008000        30   
143      276.395500    0.001000        276.392+     0.008000        30   
144      257.065200    0.001000        257.065+     0.007000        30   
145      492.388920    0.000100        492.386+     0.024000         7   

         gA(s^-1)  
0    2.140000e+09  
1    1.190000e+09  
2             NaN  
3    6.000000e+08  
4          

# 0.0333 is the sprectral resolution of the Handheld LIBS Device

Now , for the two wavelengths from two different data source , to exactly match with each other , is going to be difficult because of the spectral resolution of wavelength - for these two data source are very different from each other.

Now we know that the Spectral Resolution of Handheld LIBS Device is 0.0333. 

Now , we have a list of all wavelengths , detected for Peaks in Reference Spectra , Now for each element lets try to create a min and max of each element in the list. 'Reference_Spectra_Min_Max' is list consisting of a sublist , the sublist have a min and max value or a range of element in the list.


Eg : - A peak of 254.8 in NIST Spectra , can be found at and between [254.7668 ~ 254.8332] in the Handheld LIBS Spectra , because the resolution of wavelength in NIST Spectra and Handheld LIBS Spectra is different. 


In [7]:
Reference_Spectra_Min_Max = []
Reference_Spectra_Min_Max = [[num - 0.0332, num + 0.0332] for num in Reference_Spectra_List]     
    
print(Reference_Spectra_Min_Max)

[[213.824151, 213.890551], [334.46813899999995, 334.534539], [208.6618, 208.7282], [330.22508899999997, 330.291489], [260.82255599999996, 260.888956], [481.02000999999996, 481.08641], [280.05314599999997, 280.119546], [472.18249, 472.24889], [277.052177, 277.118577], [258.210183, 258.276583], [277.064197, 277.130597], [206.9798, 207.0462], [330.260749, 330.327149], [210.40933, 210.47573], [467.98039, 468.04679000000004], [275.61184399999996, 275.678244], [334.523739, 334.590139], [280.071801, 280.13820100000004], [271.21535, 271.28175000000005], [577.1768000000001, 577.2432], [256.953786, 257.020186], [209.65980000000002, 209.7262], [207.87491, 207.94131], [268.38266699999997, 268.449067], [260.831188, 260.89758800000004], [462.94761, 463.01401000000004], [530.8308000000001, 530.8972], [328.19935899999996, 328.265759], [208.70012, 208.76651999999999], [256.74679999999995, 256.8132], [636.2013800000001, 636.26778], [258.215754, 258.28215400000005], [254.1988, 254.2652], [577.5168, 577.5

# Noise Estimation
As we see in the Handheld LIBS Spectra , It's difficult to find or detect , what is the noise and what is the signal.

But we have an information , that the range of Spectrometer in hLIBS starts from 190nm , So we can say that the signal detected uptil 190nm could be classified as noise.

In [8]:
noise_list = [] 

noise_list = Sample_Spectra.loc[Sample_Spectra['wavelength'] < 195.0 , 'Intensities']

# RMS_intensity_noise = max(noise_list) 
RMS_intensity_noise = np.sqrt(np.mean((noise_list)**2)) #This is RMS of the noise signal

print(RMS_intensity_noise)  



243.3820626527242


In [9]:
#METHOD  - Scipy

Threshold = 5 #(Noise Intensity * Threshold) Value for signal filtering
height_2 = None
Peak_Prominenace = (Threshold * RMS_intensity_noise) # #Uses S/N ratio , as comapred to other peaks
minimum_distance_3 = None   #minimum distance between two peaks

Sample_Peak,_ = fp(Sample_Spectra.Intensities,height=height_2,prominence=Peak_Prominenace,distance=minimum_distance_3)
Selected_Sample_Spectra = Sample_Spectra.iloc[Sample_Peak] 

Sample_Spectra_Peaks = figure(title = 'Peaks in hLIBS Spectra', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Peaks.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="green" )
Sample_Spectra_Peaks.circle(Selected_Sample_Spectra.wavelength , Selected_Sample_Spectra.Intensities , color = 'red' , size = 8)

Sample_Spectra_Peaks.width = 1200
Sample_Spectra_Peaks.height = 600

print(Selected_Sample_Spectra)
show(Sample_Spectra_Peaks)

       wavelength   Intensities
392    193.066667   3002.870786
678    202.600000  22697.570546
786    206.200000  22998.613224
793    206.433333  17406.088346
810    207.000000   6054.092156
838    207.933333   4006.765727
861    208.700000  12808.214772
891    209.700000   4042.276632
901    210.033333  18085.215626
907    210.233333   7016.783465
913    210.433333   5538.549398
1016   213.866667  24522.039590
1745   238.166667   1709.017007
2106   250.200000  10317.128704
2274   255.800000  16764.673195
2397   259.900000   2198.586405
2912   277.066667   4071.098970
3002   280.066667   6709.095219
3708   303.600000   1685.986957
3817   307.233333   2994.394325
3846   308.200000   1535.346632
3878   309.266667   2423.663669
4447   328.233333  20355.935919
4508   330.266667  26272.673100
4636   334.533333  26570.052325
6400   393.333333   9676.121461
6433   394.433333   4418.568063
6484   396.133333   7527.239291
6504   396.800000   7428.622301
7281   422.700000   2152.550021
8641   4

# Wavelengths for Detected Peak(red circles above)

In [10]:
Sample_Spectra_List = []

for iteam in Selected_Sample_Spectra.wavelength:
    iteam = float(iteam)
    Sample_Spectra_List.append(iteam)
    
print(Sample_Spectra_List)


[193.06666666666592, 202.59999999999872, 206.1999999999985, 206.43333333333183, 206.9999999999985, 207.93333333333172, 208.6999999999984, 209.6999999999983, 210.03333333333163, 210.2333333333316, 210.4333333333316, 213.86666666666477, 238.16666666666336, 250.199999999996, 255.7999999999957, 259.8999999999988, 277.0666666666791, 280.0666666666815, 303.6000000000336, 307.2333333333698, 308.2000000000372, 309.26666666670474, 328.2333333333865, 330.26666666672145, 334.5333333333915, 393.3333333334383, 394.4333333334392, 396.1333333334405, 396.8000000001077, 422.70000000012834, 468.03333333349775, 471.666666666834, 472.0333333335009, 472.3000000001678, 480.33333333350754, 481.1333333335082, 492.5000000001839, 588.9666666667961, 589.5333333334622, 636.2333333334198, 656.2666666667349, 696.6000000000315, 706.7333333333556, 727.3333333333369, 738.3999999999935, 750.4333333333159, 751.4666666666483, 763.533333333304, 772.3999999999626, 794.8333333332755, 800.5999999999369, 801.4666666666028, 81

# So far so now , i have two list with me 

1) Sample_Spectra_List = Containg wavelngths of all Peaks from data of Handheld LIBS device
2) Reference_Spectra_Min_Max = Containing a range of wavelength , for all the peak detected in the Reference Sample

   

In [11]:
print(Sample_Spectra_List)

[193.06666666666592, 202.59999999999872, 206.1999999999985, 206.43333333333183, 206.9999999999985, 207.93333333333172, 208.6999999999984, 209.6999999999983, 210.03333333333163, 210.2333333333316, 210.4333333333316, 213.86666666666477, 238.16666666666336, 250.199999999996, 255.7999999999957, 259.8999999999988, 277.0666666666791, 280.0666666666815, 303.6000000000336, 307.2333333333698, 308.2000000000372, 309.26666666670474, 328.2333333333865, 330.26666666672145, 334.5333333333915, 393.3333333334383, 394.4333333334392, 396.1333333334405, 396.8000000001077, 422.70000000012834, 468.03333333349775, 471.666666666834, 472.0333333335009, 472.3000000001678, 480.33333333350754, 481.1333333335082, 492.5000000001839, 588.9666666667961, 589.5333333334622, 636.2333333334198, 656.2666666667349, 696.6000000000315, 706.7333333333556, 727.3333333333369, 738.3999999999935, 750.4333333333159, 751.4666666666483, 763.533333333304, 772.3999999999626, 794.8333333332755, 800.5999999999369, 801.4666666666028, 81

In [12]:
print(Reference_Spectra_Min_Max)

[[213.824151, 213.890551], [334.46813899999995, 334.534539], [208.6618, 208.7282], [330.22508899999997, 330.291489], [260.82255599999996, 260.888956], [481.02000999999996, 481.08641], [280.05314599999997, 280.119546], [472.18249, 472.24889], [277.052177, 277.118577], [258.210183, 258.276583], [277.064197, 277.130597], [206.9798, 207.0462], [330.260749, 330.327149], [210.40933, 210.47573], [467.98039, 468.04679000000004], [275.61184399999996, 275.678244], [334.523739, 334.590139], [280.071801, 280.13820100000004], [271.21535, 271.28175000000005], [577.1768000000001, 577.2432], [256.953786, 257.020186], [209.65980000000002, 209.7262], [207.87491, 207.94131], [268.38266699999997, 268.449067], [260.831188, 260.89758800000004], [462.94761, 463.01401000000004], [530.8308000000001, 530.8972], [328.19935899999996, 328.265759], [208.70012, 208.76651999999999], [256.74679999999995, 256.8132], [636.2013800000001, 636.26778], [258.215754, 258.28215400000005], [254.1988, 254.2652], [577.5168, 577.5

# Finally , My  idea is see , if each iteam in (Reference_Spectra_Min_Max) , matches with each element in (Sample_Spectra_List) , and then caluclate the exact matching Percentage 


In [13]:
plot_spectra_wavelength_list=[]
plot_reference_wavelength_list = []
count = 0
for cell in Reference_Spectra_Min_Max: 
    for iteam in Sample_Spectra_List:
        if iteam >= cell[0] and iteam <= cell[1]:
            count = count + 1 #for counting
            plot_spectra_wavelength_list.append(iteam)
            plot_reference_wavelength_list.append(cell[0] + 0.0333)

plot_spectra_intensities_list=[]
for iteam in plot_spectra_wavelength_list:
    Inte = Selected_Sample_Spectra.loc[Selected_Sample_Spectra['wavelength'] == iteam, 'Intensities'].iloc[0]
    plot_spectra_intensities_list.append(Inte)

#print(plot_spectra_intensities_list)
#print(plot_spectra_wavelength_list)

print('The total peaks detected were ' + str(count) + '\n' )

print('The following peaks , for NIST Reference Sample was detected  :- '  )
print(plot_reference_wavelength_list )

print('\n')
print('The Sample Peaks detected from the sample were:-')
spectra_df = pd.DataFrame({'wavelength': plot_spectra_wavelength_list, 'intensities': plot_spectra_intensities_list})
print(spectra_df)  



# MAX_Intensity = max(spectra_df.intensities)

# # ###########################################################################

# matching_row = Sample_Spectra[Sample_Spectra['Intensities'] == MAX_Intensity]
# print(matching_row)

The total peaks detected were 23

The following peaks , for NIST Reference Sample was detected  :- 
[213.857451, 334.50143899999995, 208.6951, 330.25838899999997, 280.08644599999997, 277.08547699999997, 277.097497, 207.0131, 330.294049, 210.44263, 468.01369, 334.557039, 209.69310000000002, 207.90821, 328.23265899999996, 636.2346800000001, 307.20637, 303.57797999999997, 250.19955000000002, 255.7947, 206.20021, 210.21671, 206.42276]


The Sample Peaks detected from the sample were:-
    wavelength   intensities
0   213.866667  24522.039590
1   334.533333  26570.052325
2   208.700000  12808.214772
3   330.266667  26272.673100
4   280.066667   6709.095219
5   277.066667   4071.098970
6   277.066667   4071.098970
7   207.000000   6054.092156
8   330.266667  26272.673100
9   210.433333   5538.549398
10  468.033333  25669.822658
11  334.533333  26570.052325
12  209.700000   4042.276632
13  207.933333   4006.765727
14  328.233333  20355.935919
15  636.233333  24059.847040
16  307.233333   2994

# RESULTS

In [14]:
percentage = (count / Length_Reference_Spectra_List) * 100
percentage

print (' Ananlysis of ' + element + ' was done , and the ' + str(count) +  ' number of spectral lines from the material sample ' + ' matched with the ' + str(Length_Reference_Spectra_List) + ' choosen Reference element of '+ element + ' therefore , we can say' ,  str(percentage) + '%' + ' matched as per the parameters set by the user')

 Ananlysis of Zn was done , and the 23 number of spectral lines from the material sample  matched with the 146 choosen Reference element of Zn therefore , we can say 15.753424657534246% matched as per the parameters set by the user


Find the Correponding Intensities , for the wavelengths available in "plot_spectra_wavelength_list" and also for the wavelengths available in "plot_refernce_intensities_list"

# Generate a Interactive Plot

In [15]:
p = figure(title = 'Peak Detection' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensities')
p.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="red" )
p.circle(spectra_df.wavelength, spectra_df.intensities , color = 'green' , size = 5)

p.width = 1200
p.height = 600

show(p)

